# authors: Rambod Azimi & Mahdi Moghassemi
# Date: May 2023

In [1]:
import numpy as np
import math
import pandas as pd

In [15]:
def replace_data(initial_data):
    # 

    initial_data['mainroad'] = initial_data['mainroad'].replace(['yes'], 1)
    initial_data['mainroad'] = initial_data['mainroad'].replace(['no'], 0)
    
    initial_data['guestroom'] = initial_data['guestroom'].replace(['yes'], 1)
    initial_data['guestroom'] = initial_data['guestroom'].replace(['no'], 0)

    initial_data['basement'] = initial_data['basement'].replace(['yes'], 1)
    initial_data['basement'] = initial_data['basement'].replace(['no'], 0)

    initial_data['hotwaterheating'] = initial_data['hotwaterheating'].replace(['yes'], 1)
    initial_data['hotwaterheating'] = initial_data['hotwaterheating'].replace(['no'], 0)

    initial_data['airconditioning'] = initial_data['airconditioning'].replace(['yes'], 1)
    initial_data['airconditioning'] = initial_data['airconditioning'].replace(['no'], 0)

    initial_data['prefarea'] = initial_data['prefarea'].replace(['yes'], 1)
    initial_data['prefarea'] = initial_data['prefarea'].replace(['no'], 0)

    initial_data['furnishingstatus'] = initial_data['furnishingstatus'].replace(['furnished'], 2) 
    initial_data['furnishingstatus'] = initial_data['furnishingstatus'].replace(['semi-furnished'], 1)
    initial_data['furnishingstatus'] = initial_data['furnishingstatus'].replace(['unfurnished'], 0)

    return initial_data

In [44]:
def predict(x, w, b): 
    """
    single predict using linear regression
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """
    m = x.shape[0]
    p = np.zeros(m)
    for i in range(m):
      p[i] = (np.dot(x[i,:], w[i,:]) + b[i])   

    return p 

In [17]:
initial_data = pd.read_csv('Housing.csv')
y_train = initial_data.iloc[:, 0].to_numpy()
initial_data = replace_data(initial_data)
x_train = initial_data.iloc[:, 1:].to_numpy()


[[7420    4    2 ...    2    1    2]
 [8960    4    4 ...    3    0    2]
 [9960    3    2 ...    2    1    1]
 ...
 [3620    2    1 ...    0    0    0]
 [2910    3    1 ...    0    0    2]
 [3850    3    1 ...    0    0    0]]


In [19]:
# data is stored in numpy array/matrix
print(f"x Shape: {x_train.shape}, X Type:{type(x_train)})")
print(x_train)
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
print(y_train)

x Shape: (545, 12), X Type:<class 'numpy.ndarray'>)
[[7420    4    2 ...    2    1    2]
 [8960    4    4 ...    3    0    2]
 [9960    3    2 ...    2    1    1]
 ...
 [3620    2    1 ...    0    0    0]
 [2910    3    1 ...    0    0    2]
 [3850    3    1 ...    0    0    0]]
y Shape: (545,), y Type:<class 'numpy.ndarray'>)
[13300000 12250000 12250000 12215000 11410000 10850000 10150000 10150000
  9870000  9800000  9800000  9681000  9310000  9240000  9240000  9100000
  9100000  8960000  8890000  8855000  8750000  8680000  8645000  8645000
  8575000  8540000  8463000  8400000  8400000  8400000  8400000  8400000
  8295000  8190000  8120000  8080940  8043000  7980000  7962500  7910000
  7875000  7840000  7700000  7700000  7560000  7560000  7525000  7490000
  7455000  7420000  7420000  7420000  7350000  7350000  7350000  7350000
  7343000  7245000  7210000  7210000  7140000  7070000  7070000  7035000
  7000000  6930000  6930000  6895000  6860000  6790000  6790000  6755000
  6720000  668

In [45]:
initial_w = np.zeros((x_train.shape[0], x_train.shape[1]))
# print(w.shape)
initial_b = np.zeros(x_train.shape[0])
# print(b.shape)
f_wb = predict(x_train,initial_w, initial_b)

print(f_wb.shape)


(545,)
